# 2 - 使用RNN编码器-解码器学习短语表示进行统计机器翻译

在这个关于使用PyTorch和TorchText的序列到序列模型的第二个笔记本中，我们将实现 [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078) 中的模型。该模型将在编码器和解码器中仅使用单层RNN的情况下实现改进的测试困惑度。

## 介绍

让我们先回顾一下一般的编码器-解码器模型。

![](assets/seq2seq1.png)

我们使用编码器（绿色）在嵌入的源序列（黄色）上创建上下文向量（红色）。然后，我们使用该上下文向量与解码器（蓝色）和线性层（紫色）生成目标句子。

在先前的模型中，我们使用了多层LSTM作为编码器和解码器。

![](assets/seq2seq4.png)

先前模型的一个缺点是解码器试图将大量信息压缩到隐藏状态中。在解码过程中，隐藏状态将需要包含关于整个源序列的信息，以及到目前为止已解码的所有标记的信息。通过减轻一些这种信息压缩，我们可以创建一个更好的模型！

我们还将使用GRU（门控循环单元）而不是LSTM（长短时记忆）。为什么？主要因为这就是他们在论文中所做的（该论文还引入了GRU），并且因为我们上次使用了LSTM。要了解GRU（和LSTM）与标准RNN有何不同，请查看[此链接](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)。GRU是否比LSTM更好？[研究](https://arxiv.org/abs/1412.3555)表明它们几乎相同，两者都比标准RNN更好。

## 准备数据

所有数据准备步骤都（几乎）与上次相同，因此我们将非常简要地说明每个代码块的作用。有关简要回顾，请参见上一个笔记本。

我们将导入Mindspore、spaCy和一些标准模块。


In [1]:
import mindspore as ms
import mindspore.context as context
import mindspore.nn as nn
import mindspore.ops as ops

import random
import numpy as np
import spacy
import datasets
from tqdm.notebook import tqdm
import evaluate

我们将设置所有可能的随机种子以确保结果的可重复性。

In [2]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
ms.set_seed(seed)

实例化我们的德语和英语spaCy模型。

In [3]:
dataset = datasets.load_dataset("bentrevett/multi30k")

Found cached dataset json (/root/.cache/huggingface/datasets/bentrevett___json/bentrevett--multi30k-8cca6da32304eb4d/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train_data, valid_data, test_data = dataset["train"], dataset["validation"], dataset["test"]

In [6]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

之前我们颠倒了源（德语）句子，但在我们要实现的论文中，他们没有这样做，因此我们也不会这样做。
现在编写一个用于将分词器应用于每个数据拆分中所有示例的函数，并进行一些其他处理。

此函数接受来自 `Dataset` 对象的示例，应用英语和德语 spaCy 模型的分词器，将标记列表修剪到最大长度，可选地将每个标记转换为小写，然后将序列开始和序列结束标记附加到标记列表的开头和结尾。

这个函数将与 `Dataset` 的 `map` 方法一起使用，该方法需要返回一个包含每个示例中存储输出的特征的名称的字典。由于输出特征名称 "en_tokens" 和 "de_tokens" 在示例中尚未存在（我们只有 "en" 和 "de" 特征），因此这将在每个示例中创建两个新特征。

In [7]:
def tokenize_example(
    example,
    en_nlp,
    de_nlp,
    max_length,
    lower,
    sos_token,
    eos_token
):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

使用以下方式使用 `map` 方法应用 `tokenize_example` 函数。

在这里，我们将所有序列修剪为最大长度为 1000 个标记，将每个标记转换为小写，并分别使用 `<sos>` 和 `<eos>` 作为序列的开始和结束标记。


In [8]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp, 
    "de_nlp": de_nlp, 
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}


train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

现在我们可以查看一个示例，确认已添加了两个新特征；这两个特征都是附加了序列开始/结束标记的小写字符串列表。

In [9]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

接下来，我们将为源语言和目标语言构建*词汇表*。


In [10]:
import collections
class Vocab:
    """一个词汇表的实现"""
    def __init__(self, tokens:list, min_freq=0, reserved_tokens:list=None) -> None:
        self.default_index = None
        if tokens is not None:
            # 当第一个条件满足时，就不会跳到第二个判断，避免了空列表报错的情况。
            if len(tokens)!=0 and isinstance(tokens[0], list):
                tokens = [i for line in tokens for i in line]
        else:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter=collections.Counter(tokens)
        # 按出现词频从高到低排序
        self._token_freqs = sorted(counter.items(), key=lambda x:x[1], reverse=True)
        # 通过列表,利用序号访问词元。
        self.idx_to_token = [] + reserved_tokens # 未知词元<unk>的索引为0, 保留词元排在最前
        self.token_to_idx = {
            i: k
            for k, i in enumerate(self.idx_to_token) 
        }
        
        for token, freq in self._token_freqs:
            if freq < min_freq:  # 过滤掉出现频率低于要求的词
                break
            if token not in self.token_to_idx:  
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
        
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, input_tokens):
        """输入单字串或序列, 将其全部转化为序号编码"""
        if isinstance(input_tokens, str):
            out =  self.token_to_idx.get(input_tokens, self.default_index)
            if out is None:
                raise Exception('Please call "set_default_index" before getting unknown index')
            return out
        return [self.__getitem__(token) for token in input_tokens]
    
    def __repr__(self) -> str:
        show_items = 5 if len(self) > 5 else len(self)
        out = f"<Vocab with {len(self)} tokens: "
        for i in range(show_items):
            out += f'"{self.idx_to_token[i]}", '
        out += "...>"
        return out

    def __contains__(self, token:str) -> bool:
        return token in self.idx_to_token

    def to_tokens(self, input_keys):
        """输入单s索引或序列, 将其全部转化为词元"""
        if isinstance(input_keys, int):
            return self.idx_to_token[input_keys] if input_keys < len(self) else self.idx_to_token[0]
        elif isinstance(input_keys, (list, tuple)):
            return [self.to_tokens(keys) for keys in input_keys]
        else:
            return self.idx_to_token[0]
    
    def get_itos(self):
        return self.idx_to_token
    
    def get_stoi(self):
        return self.token_to_idx
    
    def set_default_index(self, idx):
        if isinstance(idx, int):
            self.default_index = idx
        else:
            raise Exception(f"Only type int allowed, got {type(idx)}")
    
    def lookup_indices(self, input_tokens):
        return self.__getitem__(input_tokens)
    
    def lookup_tokens(self, idx):
        return self.to_tokens(idx)

创建我们的词汇表，将所有出现不到两次的标记转换为 `<unk>` 标记。

In [11]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

def build_vocab_from_iterator(tokens, min_freq, specials):
    return Vocab(tokens, min_freq, specials)

en_vocab = build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,  
)


以程序化的方式获取 `<unk>` 标记的索引，并检查我们的两个词汇表是否对未知和填充标记具有相同的索引，因为这将简化后面的一些代码。

我们还获取 `<pad>` 标记的索引，因为我们稍后将使用它。

In [12]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

使用 `set_default_index` 方法，我们可以设置在尝试获取词汇表范围之外的标记的索引时返回的值。在这种情况下，即未知标记 `<unk>` 的索引。

In [13]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

创建一个 `numericalize_example` 函数，将在数据集的 `map` 方法中使用。这将使用词汇表“数字化”（一个说法是将标记转换为索引）每个示例中的标记，并将结果返回到新的 "en_ids" 和 "de_ids" 特征中。

In [14]:
def numericalize_example(example, en_vocab:Vocab, de_vocab:Vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

应用 `numericalize_example` 函数，将我们的词汇表传递给 `fn_kwargs` 参数中的字典。

In [15]:
fn_kwargs = {
    "en_vocab": en_vocab, 
    "de_vocab": de_vocab
}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

`datasets` 库使用 `Dataset` 类为我们处理的另一件事是将特征转换为正确的类型。每个示例中的索引当前是基本的 Python 整数。但是，为了在 mindspore 中使用它们，它们需要转换为 mindspore 张量。我们可以在将它们传递到模型之前将它们转换，但是现在这样做更加方便。

`with_format` 方法将由 `columns` 参数指示的特征转换为给定的 `type`。在这里，我们将类型指定为 "numpy"（用于 mindspore），并将列指定为 "en_ids" 和 "de_ids"（我们要转换为 mindspore 张量的特征）。默认情况下，`with_format` 将删除未在传递给 `columns` 的特征列表中的特征。我们希望保留这些特征，可以使用 `output_all_columns=True`。

In [16]:
data_type = "numpy"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, 
    columns=format_columns, 
    output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type, 
    columns=format_columns, 
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type, 
    columns=format_columns, 
    output_all_columns=True,
)

是创建数据加载器。这些加载器可以进行迭代，以返回一批数据，每个批次都是一个包含数值化的英语和德语句子的字典（它们也已被填充）作为 Mindspore 张量。

In [17]:
def pad_sequence(sequences:list, padding_value:int):
    '''将序列填充到等长并返回mindspore张量'''
    # Find the length of the longest sequence in the batch
    max_length = max(len(seq) for seq in sequences)
    padded_sequences:ms.Tensor = ops.full((len(sequences), max_length), padding_value, dtype=ms.int64)
    # Copy the sequences into the padded array
    for i, seq in enumerate(sequences):
        padded_sequences[i, :len(seq)] = ms.tensor(seq).astype(np.int64)
    # 换轴，保证输出为时序优先
    padded_sequences = padded_sequences.swapaxes(0, 1)
    return padded_sequences  

def get_collate_fn(pad_index):
    
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch
    
    return collate_fn

编写使用  `DataLoader` 类创建的数据加载器的函数。

`get_data_loader` 使用 `Dataset`、批次大小、填充标记索引（用于在 `collate_fn` 中创建批次）以及一个布尔值（决定在迭代数据加载器时是否应该对示例进行洗牌）创建。




In [18]:
class DataLoader:
    def __init__(self, source, batch_size, shuffle=False, per_batch_map=None):
        self.source = source
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.per_batch_map = per_batch_map
        self.indices = np.arange(len(source))
        self.current_index = 0

        if self.shuffle:
            np.random.shuffle(self.indices)

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_index >= len(self.source):
            self.current_index = 0
            raise StopIteration

        batch_indices = self.indices[self.current_index:self.current_index + self.batch_size]
        batch_data = [self.source[int(i)] for i in batch_indices]

        if self.per_batch_map:
            batch_data = self.per_batch_map(batch_data)

        self.current_index += self.batch_size
        return batch_data
    
    def __len__(self):
        return len(self.source) // self.batch_size


def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    dataloader = DataLoader(dataset, batch_size, shuffle=shuffle, per_batch_map=collate_fn)
    
    return dataloader

In [19]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [ ]:
class GRU(nn.Cell):
    def __init__(self, embedding_dim, hidden_dim):
        super().__init__()
        self.para  = self.get_gru_params(embedding_dim, hidden_dim)
        self.state = self.init_gru_state(2, hidden_dim)
    
    def construct(self, src):
        # src = [src length, batch size]
        y, state = self.gru_forward(src, self.state, self.para)
        return hidden
    
    def get_gru_params(self, embed_size:int, num_hiddens:int):
        """获取GRU中的所有参数:
        `W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q`
        """
        num_inputs = num_outputs = embed_size
        def wwb():
            return (ops.randn((num_inputs, num_hiddens))/100,
                    ops.randn((num_hiddens, num_hiddens))/100,
                    ops.zeros((num_hiddens)),
            )
        W_xz, W_hz, b_z = wwb()
        W_xr, W_hr, b_r = wwb()
        W_xh, W_hh, b_h = wwb()
        W_hq = ops.randn((num_hiddens, num_outputs))/100
        b_q  = ops.zeros((num_outputs))
        params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
        return params
    
    def init_gru_state(self, batch_size, num_hiddens):
        """ 初始化隐变量 """
        return ops.zeros((batch_size, num_hiddens))
    
    def gru_forward(self, inputs, state, params):
        W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
        H = state
        outputs = []
        for x in inputs:
            Z = ops.sigmoid((x @ W_xz) + (H @ W_hz) + b_z)
            R = ops.sigmoid((x @ W_xr) + (H @ W_hr) + b_r)
            H_tilda = ops.tanh((x @ W_xh) + ((R * H) @ W_hh) + b_h )
            H = Z * H + (1 - Z) * H_tilda
            Y = H @ W_hq + b_q
            outputs.append(Y)
        return ops.cat(outputs, axis=0), H

## 构建 Seq2Seq 模型

### 编码器（Encoder）

编码器与先前的模型类似，多层 LSTM 被单层 GRU 替换。我们也不将 dropout 作为参数传递给 GRU，因为 dropout 是在多层 RNN 的每一层之间使用的。因为我们只有一层，如果我们尝试将 dropout 值传递给它，PyTorch 将显示警告。

关于 GRU 还要注意的一点是，它只需要并返回一个隐藏状态，没有像 LSTM 中那样的 cell 状态。

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

从上面的方程式中，看起来 RNN 和 GRU 是相同的。然而，在 GRU 内部，有一些控制信息流入和流出隐藏状态的 *门控机制*（类似于 LSTM）。再次说明，欲了解更多信息，请查看[这篇](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)博文。

编码器的其余部分应该与上个教程非常相似，它接收一个序列 $X = \{x_1, x_2, ... , x_T\}$，通过嵌入层传递它，循环计算隐藏状态 $H = \{h_1, h_2, ..., h_T\}$，并返回一个上下文向量（最终隐藏状态）$z=h_T$。

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

这与通用 seq2seq 模型的编码器相同，所有的“魔法”都发生在 GRU 内部（绿色）。

![](assets/seq2seq5.png)


In [20]:
class Encoder(nn.Cell):
    def __init__(self, input_dim, embedding_dim, hidden_dim, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        
    def construct(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, hidden = self.rnn(embedded) # no cell state in GRU!
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden

## 解码器 (Decoder)

解码器是实现与先前模型显著不同之处，我们通过一些方法减轻了信息压缩。

解码器中的 GRU 不再仅仅将嵌入目标token $d(y_t)$ 和先前隐藏状态 $s_{t-1}$ 作为输入，它还接受上下文向量 $z$。

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

请注意，这个上下文向量 $z$ 没有 $t$ 下标，这意味着我们在解码器的每个时间步骤中都重用编码器返回的相同上下文向量。

在之前的模型中，我们使用线性层 $f$ 仅使用该时间步长的顶层解码器隐藏状态 $s_t$ 预测下一个token $\hat{y}_{t+1}$，即 $\hat{y}_{t+1}=f(s_t^L)$。现在，我们还将当前token的嵌入 $d(y_t)$ 和上下文向量 $z$ 传递到线性层。

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

因此，我们的解码器现在看起来像这样：

![](assets/seq2seq6.png)

请注意，初始隐藏状态 $s_0$ 仍然是上下文向量 $z$，因此在生成第一个token时，我们实际上将两个相同的上下文向量输入到 GRU 中。

这两个更改如何减少信息压缩呢？嗯，假设解码器隐藏状态 $s_t$ 不再需要包含关于源序列的信息，因为它始终可用作输入。因此，它只需要包含有关到目前为止生成了哪些token的信息。将 $y_t$ 添加到线性层中还意味着该层可以直接看到token是什么，而不必从隐藏状态获取此信息。

然而，这个假设只是一个假设，不可能确定模型实际上是如何使用提供给它的信息的（不要听任何说法与此不同的人）。尽管如此，这是一个很好的直觉，并且结果似乎表明这些修改是一个好主意！

在实现中，我们将 $d(y_t)$ 和 $z$ 传递给 GRU，方法是将它们连接在一起，因此 GRU 的输入维度现在为 `emb_dim + hid_dim`（因为上下文向量的大小将为 `hid_dim`）。线性层将 $d(y_t), s_t$ 和 $z$ 通过将它们连接在一起传递，因此输入维度现在为 `emb_dim + hid_dim*2`。由于 GRU 只使用一个层，我们也不传递 dropout 的值给它。

在 `forward` 中，现在需要一个 `context` 参数。在 `forward` 内部，我们将 $y_t$ 和 $z$ 连接在一起，形成 `emb_con`，然后将其馈送到 GRU 中，我们将 $d(y_t)$，$s_t$ 和 $z$ 连接在一起，形成 `output`，然后将其馈送到线性层，以接收我们的预测 $\hat{y}_{t+1}$。


In [21]:
class Decoder(nn.Cell):
    def __init__(self, output_dim, embedding_dim, hidden_dim, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim + hidden_dim, hidden_dim)
        self.fc_out = nn.Dense(embedding_dim + hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(p=dropout)
        
    def construct(self, input:ms.Tensor, hidden:ms.Tensor, context:ms.Tensor):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # context = [n layers * n directions, batch size, hidden dim]
        # n layers and n directions in the decoder will both always be 1, therefore:
        # hidden = [1, batch size, hidden dim]
        # context = [1, batch size, hidden dim]
        input = input.unsqueeze(0)
        #input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, embedding dim]
        emb_con = ops.cat((embedded, context), axis = 2)
        #emb_con = [1, batch size, embedding dim + hidden dim]
        output, hidden = self.rnn(emb_con, hidden)
        # output = [seq len, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # seq len, n layers and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [1, batch size, hidden dim]
        output = ops.cat((
            embedded.squeeze(0), 
            hidden.squeeze(0), 
            context.squeeze(0)
        ),
            axis=1)
        # output = [batch size, embedding dim + hidden dim * 2]
        prediction = self.fc_out(output)
        # prediction = [batch size, output dim]
        return prediction, hidden

## Seq2Seq 模型

将编码器和解码器放在一起，我们得到：

![](assets/seq2seq7.png)

同样，在此实现中，我们需要确保编码器和解码器中的隐藏维度相同。

简要地概述所有步骤：
- 创建 `outputs` 张量以保存所有预测 $\hat{Y}$
- 将源序列 $X$ 输入编码器以接收 `context` 向量
- 将初始解码器隐藏状态设置为 `context` 向量，$s_0 = z = h_T$
- 我们使用一批 `<sos>` token作为第一个 `input`，$y_1$
- 然后我们在循环内解码：
  - 将输入token $y_t$，先前的隐藏状态 $s_{t-1}$ 和上下文向量 $z$ 插入解码器
  - 接收预测 $\hat{y}_{t+1}$ 和新的隐藏状态 $s_t$
  - 然后，我们决定是否要使用 teacher force，根据情况设置下一个输入（目标序列中的下一个真实token或最高预测的下一个token）


In [22]:
class Seq2Seq(nn.Cell):
    def __init__(self, encoder:Encoder, decoder:Decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        assert encoder.hidden_dim == decoder.hidden_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def construct(self, src:ms.Tensor, trg:ms.Tensor, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        # outputs = torch.zeros(trg_length, batch_size, trg_vocab_size)
        outputs = []
        # last hidden state of the encoder is the context
        context = self.encoder(src)
        # context = [n layers * n directions, batch size, hidden dim]
        # context also used as the initial hidden state of the decoder
        hidden = context
        # hidden = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0,:]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden state and the context state
            # receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            if len(outputs) == 0:
                outputs.append(ops.zeros(output.shape, dtype=output.dtype))
            # output = [batch size, output dim]
            # hidden = [1, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            # outputs[t] = output
            outputs.append(output)
            # get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            if self.training:
                teacher_force = random.random() < teacher_forcing_ratio
                # if teacher forcing, use actual next token as next input
                # if not, use predicted token
                input = trg[t] if teacher_force else top1
                # input = [batch size]
            else:
                input = top1
        return ops.stack(outputs, axis=0)

# 训练 Seq2Seq 模型

这个教程的其余部分与前一个非常相似。

我们初始化编码器、解码器和 Seq2Seq 模型。与之前一样，嵌入维度和使用的 dropout 量可以在编码器和解码器之间不同，但隐藏维度必须保持相同。

In [33]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    decoder_dropout,
)


model = Seq2Seq(encoder, decoder)

接下来，我们初始化参数。该论文指出，参数应该从均值为 0、标准差为 0.01 的正态分布中初始化，即 $\mathcal{N}(0, 0.01)$。

它还指出我们应该将循环参数初始化为特殊的初始化值，但为了保持简单，我们也将它们初始化为 $\mathcal{N}(0, 0.01)$。


In [34]:
def init_weights(m:nn.Cell):
    for name, param in m.parameters_and_names():
        param.set_data(ops.uniform(param.shape, ms.Tensor([-0.08]), ms.Tensor([0.08]), dtype=param.dtype))

model.apply(init_weights)

Seq2Seq<
  (encoder): Encoder<
    (embedding): Embedding<vocab_size=7853, embedding_size=256, use_one_hot=False, embedding_table=Parameter (name=encoder.embedding.embedding_table, shape=(7853, 256), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>
    (rnn): GRU<
      (rnn): _DynamicGRUCPUGPU<>
      (dropout_op): Dropout<p=0.0>
      >
    (dropout): Dropout<p=0.5>
    >
  (decoder): Decoder<
    (embedding): Embedding<vocab_size=5893, embedding_size=256, use_one_hot=False, embedding_table=Parameter (name=decoder.embedding.embedding_table, shape=(5893, 256), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>
    (rnn): GRU<
      (rnn): _DynamicGRUCPUGPU<>
      (dropout_op): Dropout<p=0.0>
      >
    (fc_out): Dense<input_channels=1280, output_channels=5893, has_bias=True>
    (dropout): Dropout<p=0.5>
    >
  >

我们打印出参数的数量。

尽管我们的编码器和解码器只有一个单层 RNN，但实际上我们拥有比上一个模型**更多**的参数。这是由于 GRU 和线性层的输入大小增加所致。然而，这不是一个显著的参数量，而且会导致训练时间的极小增加（每个时代额外增加约 3 秒）。


In [35]:
def count_parameters(model:nn.Cell):
    return sum(p.numel() for p in model.get_parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 14,219,781 trainable parameters


我们初始化优化器。


In [36]:
optimizer = nn.Adam(model.trainable_params())

我们还初始化损失函数，确保忽略 `<pad>` token上的损失。


In [37]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

然后，我们创建训练循环...


In [38]:
def forward_fn(src, trg, teacher_forcing_ratio):
    # src = [src length, batch size]
    # trg = [trg length, batch size]
    output = model(src, trg, teacher_forcing_ratio)
    # output = [trg length, batch size, trg vocab size]
    output_dim = output.shape[-1]
    output = output[1:].view(-1, output_dim)
    # output = [(trg length - 1) * batch size, trg vocab size]
    trg = trg[1:].view(-1)
    # trg = [(trg length - 1) * batch size]
    loss = criterion(output, trg.astype(ms.int32))
    return loss

grad_fn = ms.value_and_grad(forward_fn, grad_position=None, weights=model.trainable_params())


def train_fn(data_loader, optimizer, clip, teacher_forcing_ratio):
    epoch_loss = 0.
    model.set_train(True)
    for i, batch in enumerate(tqdm(data_loader)):
        src = batch["de_ids"]
        trg = batch["en_ids"]
        loss, grads = grad_fn(src, trg, teacher_forcing_ratio)
        # grads = ops.clip_by_norm(grads, max_norm=clip)
        optimizer(grads)
        epoch_loss += float(loss)

    return epoch_loss / len(data_loader)

...和评估循环，记得关闭teaching forcing。


In [39]:
def evaluate_fn(model:Seq2Seq, data_loader, criterion):
    epoch_loss = 0
    model.set_train(False)
    for i, batch in enumerate(data_loader):
        src = batch["de_ids"]
        trg = batch["en_ids"]
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        output = model(src, trg, 0) # turn off teacher forcing
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg.astype(ms.int32))
        epoch_loss += float(loss)
    return epoch_loss / len(data_loader)

然后，我们训练我们的模型，保存给出最佳(best)验证损失的参数。


In [40]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm(range(n_epochs)):
        
    train_loss = train_fn(
        train_data_loader, 
        optimizer, 
        clip, 
        teacher_forcing_ratio, 
    )
    
    valid_loss = evaluate_fn(
        model, 
        valid_data_loader, 
        criterion, 
    )

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        ms.save_checkpoint(model, f"./tut2-model.ckpt")
    
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/226 [00:00<?, ?it/s]

	Train Loss:   4.760 | Train PPL: 116.738
	Valid Loss:   5.648 | Valid PPL: 283.596


  0%|          | 0/226 [00:00<?, ?it/s]

	Train Loss:   4.263 | Train PPL:  71.009
	Valid Loss:   5.302 | Valid PPL: 200.643


...

	Train Loss:   2.269 | Train PPL:   9.666
	Valid Loss:   4.308 | Valid PPL:  74.269


  0%|          | 0/226 [00:00<?, ?it/s]

	Train Loss:   2.099 | Train PPL:   8.159
	Valid Loss:   4.253 | Valid PPL:  70.292


  0%|          | 0/226 [00:00<?, ?it/s]

	Train Loss:   1.944 | Train PPL:   6.987
	Valid Loss:   4.274 | Valid PPL:  71.792


最后，我们使用这些“best”参数在测试集上测试模型。

In [42]:
ms.load_checkpoint(f"tut2-model.ckpt", model)

test_loss = evaluate_fn(model, test_data_loader, criterion)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 4.142 | Test PPL:  62.906 |


仅看测试损失，我们获得了比先前模型更好的性能。这是这种模型架构做得不错的一个很好的迹象！缓解信息压缩似乎是前进的道路，在下一个教程中，我们将进一步扩展这一点，引入“注意力”机制。
